# LDA demo
### source: https://gist.github.com/georgehc/d2353feef7e09b4b53fc087d44f75954

In [24]:
import nltk
import json

In [26]:
content = []
for line in open('JACS.json', 'r'):
    content.append(json.loads(line))

In [9]:
Total = []
for c in content:
    ##using both title and content
    total = c['Title']
    Total.append(total)
    
#print(Total)

In [10]:
vocab_size = 1000
from sklearn.feature_extraction.text import CountVectorizer

# document frequency (df) means number of documents a word appears in
tf_vectorizer = CountVectorizer(max_df=0.95,
                                min_df=2,
                                max_features=vocab_size,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(Total)

In [11]:
num_topics = 20

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=num_topics, learning_method='online', random_state=95865)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=20, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=95865, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [12]:
import numpy as np
topic_word_distributions = np.array([topic_word_pseudocounts / np.sum(topic_word_pseudocounts)
                                     for topic_word_pseudocounts in lda.components_])

In [13]:
num_top_words = 3

print('Displaying the top %d words per topic and their probabilities within the topic...' % num_top_words)
print()

import numpy as np
for topic_idx in range(num_topics):
    print('[Topic %d]' % topic_idx)
    sort_indices = np.argsort(topic_word_distributions[topic_idx])[::-1]
    for rank in range(num_top_words):
        word_idx = sort_indices[rank]
        print('%s: %f' % (tf_vectorizer.get_feature_names()[word_idx], topic_word_distributions[topic_idx, word_idx]))
    print()


Displaying the top 3 words per topic and their probabilities within the topic...

[Topic 0]
catalyzed: 0.099926
enantioselective: 0.050113
asymmetric: 0.026763

[Topic 1]
acid: 0.061242
assisted: 0.054093
release: 0.047830

[Topic 2]
high: 0.054280
structure: 0.048515
organic: 0.039508

[Topic 3]
organic: 0.063362
metal: 0.061175
assembly: 0.057230

[Topic 4]
synthesis: 0.088919
acid: 0.035682
total: 0.027334

[Topic 5]
catalytic: 0.049506
structural: 0.044886
based: 0.042281

[Topic 6]
oxidation: 0.069529
responsive: 0.064461
absorption: 0.037167

[Topic 7]
reactions: 0.075561
chemical: 0.063651
electrochemical: 0.058867

[Topic 8]
alkenes: 0.053754
reaction: 0.053373
driven: 0.027310

[Topic 9]
transfer: 0.047278
protein: 0.035475
radical: 0.034704

[Topic 10]
imaging: 0.040989
reveals: 0.029048
binding: 0.027287

[Topic 11]
molecular: 0.103067
design: 0.037005
correction: 0.025311

[Topic 12]
light: 0.040551
oxygen: 0.038857
surface: 0.037438

[Topic 13]
formation: 0.045807
selectiv

# Another Way of doing LDA
### Source: https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

In [14]:
from nltk.corpus import stopwords 
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(entry).split() for entry in Total]

#print(doc_clean)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Maggie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Maggie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]


#mystring = mystring..decode(‘utf-8’)



In [16]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=15, id2word = dictionary, passes=50)


/Users/Maggie/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:678: RuntimeWarning: overflow encountered in exp
  expElogthetad = np.exp(Elogthetad)


In [11]:
print(ldamodel.print_topics(num_topics=15, num_words=4))

[(0, '0.007*"reaction" + 0.006*"surface" + 0.005*"catalyst" + 0.005*"co2"'), (1, '0.015*"reaction" + 0.010*"catalyst" + 0.009*"synthesis" + 0.007*"bond"'), (2, '0.005*"ligand" + 0.005*"site" + 0.004*"molecular" + 0.004*"state"'), (3, '0.007*"complex" + 0.005*"reaction" + 0.004*"material" + 0.004*"state"'), (4, '0.009*"cell" + 0.008*"imaging" + 0.005*"fluorescence" + 0.004*"probe"'), (5, '0.005*"structure" + 0.004*"state" + 0.004*"electron" + 0.004*"effect"'), (6, '0.011*"complex" + 0.009*"state" + 0.007*"reaction" + 0.006*"ligand"'), (7, '0.007*"protein" + 0.005*"rna" + 0.005*"reaction" + 0.004*"structure"'), (8, '0.010*"cell" + 0.005*"protein" + 0.005*"molecular" + 0.004*"material"'), (9, '0.006*"cofs" + 0.005*"structure" + 0.005*"organic" + 0.005*"synthesis"'), (10, '0.011*"protein" + 0.008*"cell" + 0.008*"dna" + 0.008*"surface"'), (11, '0.006*"structure" + 0.005*"framework" + 0.004*"reaction" + 0.004*"metal"'), (12, '0.018*"bond" + 0.008*"reaction" + 0.007*"intermediate" + 0.007*"co